#### Demonstrates how to load simulation results in an h5 and then pull them from orca.

In [1]:
import gc
import pandas as pd
import orca
from smartpy_sim_indicators import *

In [2]:
# location of the h5 containing sim results
# change this to wherever you have downloaded the results h5 to
sim_h5 = r'\\azsmart\AZSmartData\proj18_19\for_indicators.h5'

In [3]:
# output h5 containing indicator/aggregation results 
out_h5 = r'D:\temp\indicators_results.h5'

In [4]:
# years present w/in the h5 results
list_store_years(sim_h5)

['2020', '2025', '2030', '2035', '2040', '2045', '2050', '2055', 'base']

In [5]:
# tables present in the base year
# of these, most commonly used are buildings
list_store_tables(sim_h5, 'base')

['buildings',
 'county_emp_controls',
 'county_gq_controls',
 'county_hh_pop_controls',
 'gq_persons',
 'households',
 'jobs',
 'k12',
 'parcels',
 'persons',
 'posths',
 'seasonal_households']

In [6]:
# tables present in simulated years
list_store_tables(sim_h5, '2020')

['buildings',
 'flu_space',
 'gq_persons',
 'households',
 'jobs',
 'k12',
 'persons',
 'posths',
 'seasonal_households']

In [7]:
# note: the parcels are not modified, so they only exist in the base
#       and do not need to be loaded for simualted years
list_store_years(sim_h5, 'parcels')

['base']

In [8]:
# load all base year tables
load_tables(sim_h5, 'base')
orca.list_tables()

['buildings',
 'county_emp_controls',
 'county_gq_controls',
 'county_hh_pop_controls',
 'gq_persons',
 'households',
 'jobs',
 'k12',
 'parcels',
 'persons',
 'posths',
 'seasonal_households']

## __data model overview__

See variables.py for additional orca computed columns available.

#### ___parcels___

- Mostly contain geographic identifiers that are broadcast to other tables
- Only need to be loaded from the base

In [9]:
orca.get_table('parcels').local.head()

,city,hex_id,section_id,bg_geoid,cbd_dist,fwys_2016_dist,fwys_2019_dist,fwys_2024_dist,fwys_2030_dist,fwys_2031_dist,...,y,area,taz_2019,mpa_2019,in_mag_2019,county_2019,mpa_fullname_2019,ewc_pinal_2019,raz_2019,maz_2019
parcel_id,,,,,,,,,,,,,,,,,,,,,
1,Youngtown,8958,4700,040130716001,88969.808812,12866.192705,12866.192705,12866.192705,12866.192705,12866.192705,...,9.470856e+05,47306.515004,248,YO,yes,Maricopa County,Youngtown,West,236,24807
2,Carefree,9615,2582,040130304012,144783.120146,57372.874256,57372.874256,57372.874256,57372.874256,57372.874256,...,1.027264e+06,83347.699902,1027,CA,yes,Maricopa County,Carefree,East,208,102706
3,Phoenix,8809,5358,040131070001,37319.300742,7211.777437,7211.777437,7211.777437,7211.777437,7211.777437,...,9.230778e+05,264445.902410,3015,PH,yes,Maricopa County,Phoenix,Central,259,301505
4,Phoenix,8791,5631,040131090031,21344.412591,1599.475788,1599.475788,1599.475788,1599.475788,1599.475788,...,9.095884e+05,23998.808872,1825,PH,yes,Maricopa County,Phoenix,Central,260,182518
5,Phoenix,8829,5365,040131066002,28416.424370,8915.915734,8915.915734,8915.915734,8915.915734,8915.915734,...,9.190640e+05,72444.902618,690,PH,yes,Maricopa County,Phoenix,Central,260,69002


#### ___buildings___

- Need to be loaded every year--even if not pulling attributes explicitly--otherwise the parcel broadcast to downstream tables (e.g. households) will fail.
- Links to parcels via parcel_id column
- Typical summary attributes: residential_units, building_type_name, non_residential_sqft, job_spaces

In [10]:
orca.get_table('buildings').local

,average_value_per_unit,non_residential_sqft,year_built,building_type_name,transient_pop_in_hh,parcel_id,sqft_per_job,residential_sqft,total_fcv,transient_pop_in_hotels,transient_hh_in_hotels,transient_hh_in_hh,residential_units
building_id,,,,,,,,,,,,,
0,18731.500000,0.0,1949.0,rsf,0.0,1370963,0,1366.0,37463.0,0.0,0.0,0.0,2.0
1,55796.375000,0.0,2001.0,rsf,0.0,1370967,0,7280.0,446371.0,0.0,0.0,0.0,8.0
2,41461.101504,0.0,2016.0,mh,22.4,1370968,0,409554.0,44114612.0,0.0,0.0,8.0,1064.0
3,951988.000000,0.0,2000.0,rsf,0.0,1370973,0,6752.0,951988.0,0.0,0.0,0.0,1.0
4,642909.000000,0.0,1995.0,rsf,0.0,1370974,0,2301.0,642909.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409683,0.000000,21740.0,1970.0,ag,NaN,39785,10512,NaN,NaN,NaN,NaN,NaN,NaN
1409684,0.000000,30436.0,1970.0,ag,NaN,18570,10512,NaN,NaN,NaN,NaN,NaN,NaN
1409685,0.000000,47828.0,1970.0,ag,NaN,1146632,10512,NaN,NaN,NaN,NaN,NaN,NaN


#### ___households___

- Links to building via building_id column.
- Has a downstream dependency on persons, so if pulling person attributes need to always load households aslo. 
- Typical summary attributes: persons, income_quintile, workers

In [11]:
orca.get_table('households').to_frame(['persons', 'income', 'income_quintile', 'workers'])

,persons,income,income_quintile,workers
household_id,,,,
0,2,83104.498855,4,2
1,2,41567.274306,2,1
2,2,62456.800473,3,2
3,1,5207.881928,1,0
4,1,21163.945708,1,0
...,...,...,...,...
1743329,4,67000.000000,3,2
1743330,4,205861.798490,5,1
1743331,5,88000.000000,4,3


#### ___persons___

- Only contains persons in households
- Links to households via household_id column
- Note: if you only need a simple pop count, can just pull persons attribute from households table
- Typical summary attributes: age, race_ethnicity, education level

In [12]:
orca.get_table('persons').local

,grade_level,education,student_status,household_id,hs_grad,is_student,marriage_status,is_worker,rel_to_head,serialno,persons,race_ethnicity,sporder,grad_degree,gender,age,recent_mover,bachelors_degree
person_id,,,,,,,,,,,,,,,,,,
0,NaN,16.0,not a student,0,1,0,never married,1,head,2015000772511,1,white,1,0,male,25,0,0
1,NaN,16.0,not a student,0,1,0,never married,1,partner,2015000772511,1,white,2,0,female,30,0,0
2,NaN,16.0,not a student,70006,1,0,never married,1,head,2015000772511,1,white,1,0,male,25,0,0
3,NaN,16.0,not a student,70006,1,0,never married,1,partner,2015000772511,1,white,2,0,female,30,0,0
4,NaN,16.0,not a student,70028,1,0,never married,1,head,2015000772511,1,white,1,0,male,25,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4646696,13.0,NaN,NaN,1743331,0,1,never married,0,child,2017000225108,1,hispanic,5,0,female,16,0,0
4646697,NaN,NaN,NaN,1743332,1,0,married,0,head,2017001282293,1,white,1,0,male,69,0,0
4646698,NaN,NaN,NaN,1743332,1,0,married,0,spouse,2017001282293,1,white,2,0,female,68,0,1


#### ___gq_persons___

- Only contains persons in group quarters
- Links directly to buildings via building_id column
- Mostly just used to get total persons
- Typical summary attributes: gq_type

In [13]:
orca.get_table('gq_persons').local

,race_ethnicity,serialno,age,gender,year_added,building_id,gq_type
gq_person_id,,,,,,,
0,hispanic,2015000851161,48,male,2018,134611,gq_pop_in_nursing_homes
1,white,2014000092194,80,male,2018,134611,gq_pop_in_nursing_homes
2,white,2014000864083,93,female,2018,134611,gq_pop_in_nursing_homes
3,white,2016000838925,50,male,2018,134611,gq_pop_in_nursing_homes
4,white,2014001221657,69,male,2018,134611,gq_pop_in_nursing_homes
...,...,...,...,...,...,...,...
88345,white,2015000894204,94,female,2018,1282062,gq_pop_in_nursing_homes
88346,white,2015000894204,94,female,2018,1282062,gq_pop_in_nursing_homes
88347,white,2017001464302,75,female,2018,1282062,gq_pop_in_nursing_homes


#### ___seaonal_households___

- Seasonal households and persons
- Links directly to buildings via building_id column
- Since we have no information on the attributes of seasonal persons, no need for additional table
- Typical summary attributes: persons

In [14]:
orca.get_table('seasonal_households').local

,src_building_id,persons,year_added,building_id
seasonal_household_id,,,,
0,82695,1,2018,82695
1,85677,1,2018,85677
2,1892,1,2018,1892
3,6855,1,2018,6855
4,71634,1,2018,71634
...,...,...,...,...
103373,858176,2,2018,858176
103374,743072,2,2018,743072
103375,1132257,2,2018,1132257


## __example indicator generation__

Typically we: 

- Define a simple function that acts on the provided geography

- Write a loop that iterates through the desired years:

    1.) Load tables for the given year

    2.) Get the summary for that year
    
- Compile the results in some way
    
    1.) Compile as columns, e.g. pop_2020, pop_2030, ...
    
    2.) Compile as rows, 1 row for each year

In [15]:
def get_summary(by):
    """
    Simple aggregation function.
    
    Parameters: 
    ----------
    by: str or list of str
        The columns to groupby.
    
    Returns:
    --------
    pandas.DataFrame
    
    """
    
    if not isinstance(by, list):
        by = [by]
    
    # pull tables
    pers = orca.get_table('persons').to_frame()
    j = orca.get_table('jobs').to_frame(by)
    gq = orca.get_table('gq_persons').to_frame(by)
    seas = orca.get_table('seasonal_households').to_frame(by + ['persons'])
    
    # groupbys
    sum_tab = pd.concat([
        pers.groupby(by).size().to_frame('hh_pop'),
        gq.groupby(by).size().to_frame('gq_pop'),
        seas.groupby(by)['persons'].sum().to_frame('seas_persons'),
        j.groupby(by).size().to_frame('jobs')
    ], axis=1).fillna(0)
    
    # get the total resident pop (hh pop + gq pop)
    sum_tab['total_pop'] = sum_tab['hh_pop'] + sum_tab['gq_pop']
    
    return sum_tab

In [16]:
# tables we need to process for the aggregation
tabs_to_process = [
    'buildings',
    'households',
    'persons',
    'seasonal_households',
    'jobs'
]

In [17]:
# generate year-based indicators by mpa (Municpal Planning Area) and county
mpa_sums = get_indicators(
    sim_h5,
    [2020, 2030, 2040],
    tabs_to_process,
    ['county', 'mpa'],
    get_summary
)
mpa_sums.keys()

on year: 2020...
on year: 2030...
on year: 2040...


dict_keys([2020, 2030, 2040])

In [18]:
# indicators compiled as columns
compile_to_cols(mpa_sums, collapse_row_idx=False).head()

hh_pop_2020  gq_pop_2020  seas_persons_2020  jobs_2020  \
county          mpa                                                           
Maricopa County AJ           345          0.0              217.0       18.0   
                AV         86528        166.0             1490.0    23164.0   
                BU         91889       5664.0             4768.0    26920.0   
                CA          3750         51.0             1036.0     1623.0   
                CC          6020          0.0              284.0     2352.0   

                     total_pop_2020  hh_pop_2030  gq_pop_2030  \
county          mpa                                             
Maricopa County AJ            345.0          347          0.0   
                AV          86694.0       101560        166.0   
                BU          97553.0       179799       5664.0   
                CA           3801.0         4062         51.0   
                CC           6020.0         6524          0.0   

                     seas_persons_2030  jobs_2030  total_pop_2030  \
county          mpa                                                 
Maricopa County AJ               217.0       15.0           347.0   
                AV              1984.0    30406.0        101726.0   
                BU              7805.0    42893.0        185463.0   
                CA              1038.0     2102.0          4113.0   
                CC               296.0     2662.0          6524.0   

                     hh_pop_2040  gq_pop_2040  seas_persons_2040  jobs_2040  \
county          mpa                                                           
Maricopa County AJ           347          0.0              217.0       15.0   
                AV        111650        166.0             2223.0    36222.0   
                BU        297588       5664.0            13000.0    64524.0   
                CA          4134         51.0             1040.0     2395.0   
                CC          7031          0.0              296.0     2879.0   

                     total_pop_2040  
county          mpa                  
Maricopa County AJ            347.0  
                AV         111816.0  
                BU         303252.0  
                CA           4185.0  
                CC           7031.0

In [19]:
# indicators compiled as temporal rows
compile_to_rows(mpa_sums, collapse_row_idx=False).head()

year  hh_pop  gq_pop  seas_persons     jobs  total_pop
county          mpa                                                        
Maricopa County AJ   2020     345     0.0         217.0     18.0      345.0
                AV   2020   86528   166.0        1490.0  23164.0    86694.0
                BU   2020   91889  5664.0        4768.0  26920.0    97553.0
                CA   2020    3750    51.0        1036.0   1623.0     3801.0
                CC   2020    6020     0.0         284.0   2352.0     6020.0

#### ___compute indicators across several groupings___

In [20]:
# things we want to aggregate by
by_items = {
    'mpa': ['mpa', 'county'],
    'city': ['city', 'county'],
    'mazes': 'maz',
    'tazes': 'taz',
    'sections': 'section_id',
    'hex': 'hex_id',
}

In [22]:
# get the sums
many_sums = get_indicators(
    sim_h5,
    [2018] + list(range(2020, 2056, 5)),
    tabs_to_process,
    by_items,
    get_summary
)
many_sums.keys()

on year: 2018...
on year: 2020...
on year: 2025...
on year: 2030...
on year: 2035...
on year: 2040...
on year: 2045...
on year: 2050...
on year: 2055...


dict_keys(['mpa', 'city', 'mazes', 'tazes', 'sections', 'hex'])

In [23]:
# compile as columns
as_cols = {}
for k, v in many_sums.items():
    as_cols[k] = compile_to_cols(v)
    
as_cols['tazes']

,hh_pop_2018,gq_pop_2018,seas_persons_2018,jobs_2018,total_pop_2018,hh_pop_2020,gq_pop_2020,seas_persons_2020,jobs_2020,total_pop_2020,...,hh_pop_2050,gq_pop_2050,seas_persons_2050,jobs_2050,total_pop_2050,hh_pop_2055,gq_pop_2055,seas_persons_2055,jobs_2055,total_pop_2055
taz,,,,,,,,,,,,,,,,,,,,,
101.0,390.0,0.0,83.0,50.0,390.0,426.0,0.0,85.0,55.0,426.0,...,621.0,0.0,85.0,171.0,621.0,638.0,0.0,85.0,180.0,638.0
102.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,118.0,0.0,0.0,0.0,0.0,28.0,0.0
103.0,24.0,0.0,0.0,7.0,24.0,24.0,0.0,0.0,6.0,24.0,...,24.0,0.0,0.0,195.0,24.0,24.0,0.0,0.0,326.0,24.0
104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,...,0.0,0.0,0.0,72.0,0.0,0.0,0.0,0.0,125.0,0.0
106.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,6.0,353.0,4.0,36.0,0.0,32.0,672.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444.0,114.0,0.0,18.0,13.0,114.0,119.0,0.0,18.0,11.0,119.0,...,865.0,0.0,42.0,210.0,865.0,1310.0,0.0,44.0,283.0,1310.0
3445.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1180.0,0.0,39.0,144.0,1180.0,1547.0,0.0,40.0,47.0,1547.0
3446.0,1038.0,0.0,22.0,40.0,1038.0,1040.0,0.0,22.0,54.0,1040.0,...,1147.0,0.0,22.0,60.0,1147.0,1149.0,0.0,22.0,68.0,1149.0


In [24]:
# compile as rows
as_rows = {}
for k, v in many_sums.items():
    as_rows[k] = compile_to_rows(v)

as_rows['hex']

,year,hh_pop,gq_pop,seas_persons,jobs,total_pop
hex_id,,,,,,
0.0,2018,50.0,0.0,0.0,1.0,50.0
488.0,2018,5.0,0.0,0.0,0.0,5.0
523.0,2018,14.0,0.0,0.0,0.0,14.0
553.0,2018,5.0,0.0,0.0,0.0,5.0
559.0,2018,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...
18729.0,2055,3.0,0.0,0.0,1.0,3.0
18761.0,2055,4.0,0.0,0.0,0.0,4.0
18953.0,2055,2.0,0.0,0.0,0.0,2.0


In [25]:
# write result to h5
with pd.HDFStore(out_h5, mode='w', complevel=1) as s:
    for k in many_sums.keys():
        s['by_col/{}'.format(k)] = as_cols[k]
        s['by_row/{}'.format(k)] = as_rows[k]

In [26]:
s = pd.HDFStore(out_h5, mode='r')

In [27]:
s.keys()

['/by_row/city',
 '/by_row/hex',
 '/by_row/mazes',
 '/by_row/mpa',
 '/by_row/sections',
 '/by_row/tazes',
 '/by_col/city',
 '/by_col/hex',
 '/by_col/mazes',
 '/by_col/mpa',
 '/by_col/sections',
 '/by_col/tazes']

In [28]:
s['/by_col/tazes']

,hh_pop_2018,gq_pop_2018,seas_persons_2018,jobs_2018,total_pop_2018,hh_pop_2020,gq_pop_2020,seas_persons_2020,jobs_2020,total_pop_2020,...,hh_pop_2050,gq_pop_2050,seas_persons_2050,jobs_2050,total_pop_2050,hh_pop_2055,gq_pop_2055,seas_persons_2055,jobs_2055,total_pop_2055
taz,,,,,,,,,,,,,,,,,,,,,
101.0,390.0,0.0,83.0,50.0,390.0,426.0,0.0,85.0,55.0,426.0,...,621.0,0.0,85.0,171.0,621.0,638.0,0.0,85.0,180.0,638.0
102.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,118.0,0.0,0.0,0.0,0.0,28.0,0.0
103.0,24.0,0.0,0.0,7.0,24.0,24.0,0.0,0.0,6.0,24.0,...,24.0,0.0,0.0,195.0,24.0,24.0,0.0,0.0,326.0,24.0
104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,...,0.0,0.0,0.0,72.0,0.0,0.0,0.0,0.0,125.0,0.0
106.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,6.0,353.0,4.0,36.0,0.0,32.0,672.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444.0,114.0,0.0,18.0,13.0,114.0,119.0,0.0,18.0,11.0,119.0,...,865.0,0.0,42.0,210.0,865.0,1310.0,0.0,44.0,283.0,1310.0
3445.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1180.0,0.0,39.0,144.0,1180.0,1547.0,0.0,40.0,47.0,1547.0
3446.0,1038.0,0.0,22.0,40.0,1038.0,1040.0,0.0,22.0,54.0,1040.0,...,1147.0,0.0,22.0,60.0,1147.0,1149.0,0.0,22.0,68.0,1149.0


In [30]:
list(s['/by_col/tazes'].columns)

['hh_pop_2018',
 'gq_pop_2018',
 'seas_persons_2018',
 'jobs_2018',
 'total_pop_2018',
 'hh_pop_2020',
 'gq_pop_2020',
 'seas_persons_2020',
 'jobs_2020',
 'total_pop_2020',
 'hh_pop_2025',
 'gq_pop_2025',
 'seas_persons_2025',
 'jobs_2025',
 'total_pop_2025',
 'hh_pop_2030',
 'gq_pop_2030',
 'seas_persons_2030',
 'jobs_2030',
 'total_pop_2030',
 'hh_pop_2035',
 'gq_pop_2035',
 'seas_persons_2035',
 'jobs_2035',
 'total_pop_2035',
 'hh_pop_2040',
 'gq_pop_2040',
 'seas_persons_2040',
 'jobs_2040',
 'total_pop_2040',
 'hh_pop_2045',
 'gq_pop_2045',
 'seas_persons_2045',
 'jobs_2045',
 'total_pop_2045',
 'hh_pop_2050',
 'gq_pop_2050',
 'seas_persons_2050',
 'jobs_2050',
 'total_pop_2050',
 'hh_pop_2055',
 'gq_pop_2055',
 'seas_persons_2055',
 'jobs_2055',
 'total_pop_2055']

In [31]:
s['/by_row/sections']

,year,hh_pop,gq_pop,seas_persons,jobs,total_pop
section_id,,,,,,
804.0,2018,0.0,0.0,0.0,20.0,0.0
826.0,2018,4.0,0.0,0.0,0.0,4.0
903.0,2018,14.0,0.0,6.0,0.0,14.0
904.0,2018,110.0,0.0,29.0,4.0,110.0
908.0,2018,140.0,3.0,25.0,97.0,143.0
...,...,...,...,...,...,...
16426.0,2055,8.0,0.0,0.0,0.0,8.0
16444.0,2055,0.0,0.0,0.0,82.0,0.0
16445.0,2055,4.0,0.0,0.0,0.0,4.0


In [32]:
s.close()